## Set the computations 

In [1]:
# Import modules 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
# Set visuales 
coloors = ['#739354', '#875590', '#7dd3ec','#f5c9d0' , '#7dd3ec']

In [3]:
# read major final demand sectors 
sectors = pd.read_excel('../Data/Meta.xlsx', sheet_name = 'Data')
sectors.head(3)

Eora_classification Sector  Industry_category
0           Agriculture   Food                  1
1               Fishing   Food                  1
2  Mining and Quarrying  Goods                  1

## 1. CES data

In [4]:
# Set general path 
ces_path = '../Data/CES/'
# Read the CES data 
y = pd.read_csv(ces_path + 'urban.ces.csv', index_col = 'City')
y.head(3)

Category Country iso3    0    1    2           3           4  \
City                                                                         
Abidjan           LMIC          CIV  0.0  0.0  0.0  356.394219   45.330338   
Addu              UMIC          MDV  0.0  0.0  0.0  813.932292  116.276042   
Agneby-Tiassa     LMIC          CIV  0.0  0.0  0.0  327.301433   44.229212   

                 5    6    7  ...   16          17          18          19  \
City                          ...                                            
Abidjan        0.0  0.0  0.0  ...  0.0    0.000000   89.879362   51.582868   
Addu           0.0  0.0  0.0  ...  0.0  116.276042  377.864583  232.552083   
Agneby-Tiassa  0.0  0.0  0.0  ...  0.0    0.000000  110.171241   58.703749   

                20   21          22   23          24   25  
City                                                       
Abidjan        0.0  0.0   24.305724  0.0   61.741964  0.0  
Addu           0.0  0.0  436.002604  0.0  174.414062  0.0  
Agneby-Tiassa  0.0  0.0   59.507327  0.0    6.641163  0.0  

[3 rows x 28 columns]

## 2. Compute the TIV ($F$)

For this anlysis we're using [Eora26](https://worldmrio.com/eora26/) (v199.82)

In [5]:
# Read modules 
import numpy as np
import pandas as pd
import os
import re

We need to compute the the Total Intensity Vector $(F)$ for these countries: *['CIV', 'MDV', 'MEX', 'DZA', 'LKA', 'CHN', 'BGD', 'ALB', 'ZWE','EGY', 'ZAF', 'MAR', 'NAM', 'BWA', 'VNM', 'ZMB', 'MNE', 'ASM', 'CRI', 'GEO', 'TUN']*

To compute the the TIF, we need to preform the following operations: 
- compute the Xout 
- compute the direct intensity vector (f) 
- comput technical coefficient matrix
- compute Wassily Leontief inverse 
- compute the Total intesnity verctor 

In [8]:
# set paths 
bp_path = '../Data/Eora/Eora_BP/'

# Read the index 
index = [file for file in os.listdir(bp_path)]
index

containers = {}
for i in index:
    path = [file for file in os.listdir(bp_path + i) if any(map(str.isdigit, file))]
    print(f' \n i == {i} \n')
    container = {}
    containers[i] = container
    for m in path:
        print(f'm == {m}')
        read = pd.read_csv(bp_path + i + '/' + m, delimiter = '\t', header = None)
        container[m.split('_')[-1].split('.')[0]] = read

 
 i == Eora26_2009_bp 

m == Eora26_2009_bp_FD.txt
m == Eora26_2009_bp_Q.txt
m == Eora26_2009_bp_QY.txt
m == Eora26_2009_bp_T.txt
m == Eora26_2009_bp_VA.txt
 
 i == Eora26_2010_bp 

m == Eora26_2010_bp_FD.txt
m == Eora26_2010_bp_Q.txt
m == Eora26_2010_bp_QY.txt
m == Eora26_2010_bp_T.txt
m == Eora26_2010_bp_VA.txt
 
 i == Eora26_2011_bp 

m == Eora26_2011_bp_FD.txt
m == Eora26_2011_bp_Q.txt
m == Eora26_2011_bp_QY.txt
m == Eora26_2011_bp_T.txt
m == Eora26_2011_bp_VA.txt
 
 i == Eora26_2012_bp 

m == Eora26_2012_bp_FD.txt
m == Eora26_2012_bp_Q.txt
m == Eora26_2012_bp_QY.txt
m == Eora26_2012_bp_T.txt
m == Eora26_2012_bp_VA.txt
 
 i == Eora26_2013_bp 

m == Eora26_2013_bp_FD.txt
m == Eora26_2013_bp_Q.txt
m == Eora26_2013_bp_QY.txt
m == Eora26_2013_bp_T.txt
m == Eora26_2013_bp_VA.txt
 
 i == Eora26_2014_bp 

m == Eora26_2014_bp_FD.txt
m == Eora26_2014_bp_Q.txt
m == Eora26_2014_bp_QY.txt
m == Eora26_2014_bp_T.txt
m == Eora26_2014_bp_VA.txt
 
 i == Eora26_2015_bp 

m == Eora26_2015_bp_FD.txt


In [295]:
## check the Q-values index
Q = pd.read_csv('../Data/Eora/Eora_BP/Eora26_2009_bp/Eora26_2009_bp_Q.txt', delimiter = '\t', header = None)
I = pd.read_csv('../Data/Eora/Eora_BP/Eora26_2009_bp/labels_Q.txt', delimiter = '\t', header = None)
lo = pd.concat([I, Q], axis = 1)
lo.iloc[2501,]

0       WFN: Total water footprint (Mm3/yr) - Grey
1                                            Total
2                                              NaN
0                                            335.7
1                                          0.25441
                           ...                    
4910                                        22.022
4911                                           0.0
4912                                           0.0
4913                                           0.0
4914                                             0
Name: 2501, Length: 4918, dtype: object

In [294]:
Blue = {}
print('Blue water footprint')
# set the computation code 
for i in containers.keys():
    # print('---')
    # print(i)
    # print('---')
    for m in containers.get(i):
        """ interate over the dic and return the TIV for each of the years"""
        np.seterr(divide='ignore', invalid='ignore')
        xout = np.empty((4915,1))
        np.add(np.add.reduce(np.array(containers.get(i).get('T')), axis =1)[:, np.newaxis], np.add.reduce(np.array(containers.get(i).get('FD')), axis =1)[:, np.newaxis], out = xout)
        # compute direct intensity vector (f) 
        f = np.zeros_like(xout)
        np.divide(np.array(containers.get(i).get('Q'))[2500,:][:, np.newaxis], xout, out=f, where= xout != 0)
        # comput technical coefficient matrix 
        A = np.empty((4915, 4915))
        np.divide(containers.get(i).get('T'), xout, out = A)
        A = np.array(pd.DataFrame(A).fillna(0))
        # creating identity matrix 
        I = np.eye(4915)
        # compute the Wassily Leontief inverse 
        L = np.linalg.inv((I - A)) 
        # compute the Total intesnity verctor 
        F = L.dot(f)
        Blue[i.split('_')[1]] = F

---
Eora26_2009_bp
---
---
Eora26_2010_bp
---
---
Eora26_2011_bp
---
---
Eora26_2012_bp
---
---
Eora26_2013_bp
---
---
Eora26_2014_bp
---
---
Eora26_2015_bp
---


In [ ]:
Grey = {}
# set the computation code 
print('Grey water footprint')
for i in containers.keys():
    print('---')
    print(i)
    print('---')
    for m in containers.get(i):
        """ interate over the dic and return the TIV for each of the years"""
        np.seterr(divide='ignore', invalid='ignore')
        xout = np.empty((4915,1))
        np.add(np.add.reduce(np.array(containers.get(i).get('T')), axis =1)[:, np.newaxis], np.add.reduce(np.array(containers.get(i).get('FD')), axis =1)[:, np.newaxis], out = xout)
        # compute direct intensity vector (f) 
        f = np.zeros_like(xout)
        np.divide(np.array(containers.get(i).get('Q'))[2501,:][:, np.newaxis], xout, out=f, where= xout != 0)
        # comput technical coefficient matrix 
        A = np.empty((4915, 4915))
        np.divide(containers.get(i).get('T'), xout, out = A)
        A = np.array(pd.DataFrame(A).fillna(0))
        # creating identity matrix 
        I = np.eye(4915)
        # compute the Wassily Leontief inverse 
        L = np.linalg.inv((I - A)) 
        # compute the Total intesnity verctor 
        F = L.dot(f)
        Grey[i.split('_')[1]] = F

In [305]:
## import the index of countries & sectors 
index = pd.read_csv('../Data/Eora/Eora_BP/Eora26_2009_bp/labels_T.txt', delimiter = '\t', header = None).iloc[:,:-1]
index.columns = ['country', 'iso3', 'industries', 'sector']
index.head(3)

country iso3  industries                sector
0  Afghanistan  AFG  Industries           Agriculture
1  Afghanistan  AFG  Industries               Fishing
2  Afghanistan  AFG  Industries  Mining and Quarrying

In [306]:
## add the countries and the sectors names 
for i in Blue.keys():
    Blue[i] = pd.concat([index, pd.DataFrame(Blue.get(i))], axis = 1)
    
for i in Grey.keys():
    Grey[i] = pd.concat([index, pd.DataFrame(Grey.get(i))], axis = 1)   

In [424]:
# get the TIV for each country & year 
info = pd.read_csv('../infos.csv')
info.head(3)

country Eora_country  Year
0  Algeria          DZA  2011
1  Tunisia          TUN  2010
2  Morocco          MAR  2014

In [516]:
## get the data for each year & country 
Bleu_F = {}
for i in info['Eora_country'].unique():
    Year = info[info['Eora_country'] == i].Year.item()
    Bleu_F[i] = Blue.get(f"{Year}").query(f'iso3 == "{i}"')[['iso3', 'sector', 0]]
    
Grey_F = {}
for i in info['Eora_country'].unique():
    Year = info[info['Eora_country'] == i].Year.item()
    Grey_F[i] = Grey.get(f"{Year}").query(f'iso3 == "{i}"')[['iso3', 'sector', 0]]

DZA
2011
TUN
2010
MAR
2014
ALB
2015
BGR
2015
BGD
2010
BWA
2015
CRI
2013
CIV
2013
IDN
2012
MDV
2015
MYS
2014
MNE
2015
NAM
2015
ZAF
2015
LKA
2015
UKR
2015
VNM
2012
ZMB
2015
WSM
2015
GEO
2015
MEX
2015
ZWE
2012
CHN
2014


In [582]:
## get the footprints 

Bleu_foot = {}

for name in Bleu_F.keys(): 
    city = np.array(y[y['Country iso3'] == f'{name}'].iloc[:,2:].T)
    TIF = np.array(Bleu_F.get(f'{name}')[[0]])
    com = pd.DataFrame(city * TIF)
    com.columns = y[y['Country iso3'] == f'{name}'].iloc[:,2:].T.columns   
    Bleu_foot[name] = com
    
Grey_foot = {}

for name in Grey_F.keys(): 
    city = np.array(y[y['Country iso3'] == f'{name}'].iloc[:,2:].T)
    TIF = np.array(Grey_F.get(f'{name}')[[0]])
    com = pd.DataFrame(city * TIF)
    com.columns = y[y['Country iso3'] == f'{name}'].iloc[:,2:].T.columns   
    Grey_foot[name] = com